# Практика
Используемые библиотеки

In [50]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [192]:
#url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
#s = requests.get(url).content
#df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [ ]:
'''Подозрительная единица измерения для  Показателя  Цветность .  Градусы больше подходят
к измерению температуры.  Оставим данную строку , но ее анализ не будем учитывать в резутирующем заключении
так как требуется перепроверка. Значение для данного показателя примем равным нулю.''' 

In [209]:
def parse_normativ(normativ):
    normativ = normativ[0]
    normativ = normativ.replace(',','.')
    lim = re.search('не[ ]+более' , normativ.lower())
    between = re.search('в[ ]+пределах' , normativ.lower())
    tval = re.search('((\d+[.]\d+)|(\d+))-((\d+[.]\d+)|(\d+))', normativ)
    oval = re.search('(\d+[.]\d+)|(\d+)', normativ)
    if lim :
        if tval:            
            return [0, float(tval[0].split(sep='-')[1])]
        elif oval:
            return [0, float(oval[0])]
        else:
            return ['None','None']

    elif between :
        if tval:
            return [float(tval[0].split(sep='-')[0]), float(tval[0].split(sep='-')[1])]
        elif oval:
            return [0, float(oval[0])]
        else:
            return ['None','None']
    else:
        return ['None','None']


In [204]:
#заполним в отдельных колонках диапазоны значений для показателей
df[['min','max']] = df[['Норматив']].apply(lambda x: parse_normativ(x), axis=1, result_type='expand')
#приведем результаты измерений к единому формату
df['res'] = df['Результат анализа'].apply(lambda x : float(x) if re.search('[0-9,.]+',x) else float(0) )

In [229]:
#Заполним новую колонку Заключение с оценкой показателя  Ниже , Выше норматива или в пределах нормы
df['Заключение'] = 'Показатель в норме'
df.loc[df.res > df['max'],'Заключение'] = 'Превышает норматив'
df.loc[df.res < df['min'],'Заключение'] = 'Показатель ниже норматива'

In [231]:
#Добавим индекс по Показателю
df.set_index('Показатель')

,Единица измерений,Результат анализа,Норматив,min,max,res,Заключение
Показатель,,,,,,,
pH,единицы pH,8.4,в пределах 6-9,6.0,9.0,8.400,Показатель в норме
Запах,баллы,1,не более 2-3,0.0,3.0,1.000,Показатель в норме
Цветность,градусы,б/цвета,не более 30,0.0,30.0,0.000,Показатель в норме
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,7.0,10.0,9.200,Показатель в норме
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",0.0,1.5,0.420,Показатель в норме
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0.0,3.3,0.017,Показатель в норме
Нитраты (по NO3),мг/дм3,24,не более 45,0.0,45.0,24.000,Показатель в норме
Фосфаты (P),мг/дм3,0.36,"не более 3,5",0.0,3.5,0.360,Показатель в норме
Хлориды (Cl),мг/дм3,200,не более 350,0.0,350.0,200.000,Показатель в норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [232]:
import random

#Количество испытаний
N = 1000

In [255]:
#апельсинов
oranges = 5

#яблок
apples = 4

#укладываем в коробку
box = [ 'O' if x <= 5 else 'A' for x in range(1,10) ]


In [289]:
#подсчитаем количество апельсинов в наугад выбранных 3 фруктах
def get_three(box):
    random.shuffle(box)
    cnt = 0
    for el in box[:3]:
        if el == 'O':
            cnt+=1
    return cnt



In [314]:
# подсчитаем количество успешных случаев, когда все три фрукта были апельсинами.
good_attempts = 0
for i in range(N):
    if get_three(box) == 3 :
        good_attempts += 1
    
print( 'Вероятность достать наугад 3 апельсина из коробки \nс 5 апельсинами и 4 яблоками \n равна : {:.4f}'.format(good_attempts / N))
    


Вероятность достать наугад 3 апельсина из коробки 
с 5 апельсинами и 4 яблоками 
 равна : 0.1180


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [319]:
# 7 стандартных + 3 нестандартных
# разложим детали на линии в случайном порядке

line = [ 'good' if x < 7 else 'bad' for x in range(10)]


In [321]:
random.shuffle(line)
line

['good', 'good', 'bad', 'good', 'bad', 'good', 'good', 'good', 'good', 'bad']

In [322]:
# если первая детяль бракованная , а вторая стандартная то такую попытку посчитаем за 1

def second(line):
    if  line[0] == 'bad' and line[1] == 'good':
        return 1
    else:
        return 0

In [334]:
N = 1000
cnt = 0

for i in range(N):
    random.shuffle(line)
    cnt += second(line)

print( 'Вероятность того что первая нестандартная , \nа вторая стандартная деталь равна : {:.4f}'.format(cnt / N))

Вероятность того что первая нестандартная , 
а вторая стандартная деталь равна : 0.2240
